In [1]:
import sys

sys.path.append("../")

from analysis.scalar import HLFAnalysis
from models.nn import PaeBuilder

import tensorflow as tf
import numpy as np
SEED = 42
np.random.seed(SEED) 
tf.random.set_seed(SEED)

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
from utils import load_json, dump_json

from loaders.LHCO import LhcoRnDLoader

loader = LhcoRnDLoader.from_json("../configs/loader/default_rnd.json")
dataset_cfg = load_json("../configs/loader/default_dataset.json")
dataset = loader.make_full_dataset(**dataset_cfg)

In [4]:
import tensorflow.keras as tfk
import tensorflow as tf

from models.autoencoder import DenseAutoencoder
from models.flows import CondMAF, MAF

builder = PaeBuilder()

pae, ae_train, nf_train = builder.from_json('../configs/pae/trial.json')

In [5]:
task = HLFAnalysis(pae, dataset=dataset)


In [6]:
task.reweighting(estimator='gmm', fit_key='mjj_train')

In [7]:
task.make_cond_inputs(['mjj_train', 'mjj_test', 'mjj_valid'])
task.train(ae_train,nf_train)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Epoch 1/60
800/800 [==============================] - 6s 6ms/step - loss: 678.4298 - val_loss: 507.2573
Epoch 2/60
800/800 [==============================] - 1s 2ms/step - loss: 558.2496 - val_loss: 467.9574
Epoch 3/60
800/800 [==============================] - 1s 2ms/step - loss: 477.1656 - val_loss: 457.3330
Epoch 4/60
800/800 [==============================] - 1s 2ms/step - loss: 456.0481 - val_loss: 423.0026
Epoch 5/60
800/800 [==============================] - 1s 2ms/step - loss: 477.6360 - val_loss: 405.0583
Epoch 6/60
800/800 [==============================] - 1s 2ms/step - loss: 410.1052 - val_loss: 398.5220
Epoch 7/60
800/800 [==============================] - 1s 2ms/step - loss: 398.5087 - val_loss: 392.1498
Epoch 8/60
800/800 [==============================] - 1s 2ms/step - loss: 410.9281 - val_loss: 384.7665
Epoch 9/60
800/800 [===============

In [12]:
r = task.evaluate(prc = 90)

In [13]:
q = {
 'id': [],
 'js_div_train': [],
 'js_div_test': [],
 'sig_eff': [],
 'bkg_rej': [],
 'auc': []}

In [14]:
for key in r.keys():
    q[key].append(r[key])

q

{'js_div_train': [0.06687183895026455],
 'js_div_test': [0.10502881495634947],
 'sig_eff': [0.13635],
 'bkg_rej': [0.93635],
 'auc': [0.7178493959000001]}

In [10]:
task.plot_training()

In [11]:
task.plot_latent_space()